In [77]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE, RMSE
from epftoolbox.models import LEAR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from datetime import datetime as dt

In [78]:
PJM = pd.read_csv('PJMeval.csv', index_col=[0])
NP = pd.read_csv('NPeval.csv', index_col=[0])
FR = pd.read_csv('FReval.csv', index_col=[0])
NGBoost = pd.read_csv('forecast_ng.csv', index_col=[0])


df = pd.read_csv('df_test.csv', index_col=[0])


# PJM LEAR, DNN, NGBOOST

In [79]:
PJM

,Real price,DNN 1,DNN 2,DNN 3,DNN 4,DNN Ensemble,LEAR 56,LEAR 84,LEAR 1092,LEAR 1456,LEAR Ensemble
27/12/2016 00:00,19.113045,19.706123,19.040689,19.661985,18.857361,19.316540,20.1291,20.7366,19.1060,18.9901,19.740450
27/12/2016 01:00,18.042768,18.578808,18.628008,18.581722,17.804329,18.398217,19.5229,19.5459,18.2766,18.4639,18.952325
27/12/2016 02:00,17.587395,17.405716,17.954073,17.564003,16.387474,17.327816,19.0085,18.9345,17.4325,17.5918,18.241825
27/12/2016 03:00,17.399361,16.994461,17.322933,16.941095,15.673610,16.733025,18.9757,19.2347,17.4400,17.5345,18.296225
27/12/2016 04:00,17.804108,18.000519,18.452641,17.457172,15.920693,17.457756,19.3679,19.8770,18.1750,18.1798,18.899925
...,...,...,...,...,...,...,...,...,...,...,...
26/01/2017 19:00,32.532418,35.463852,34.982365,33.913315,33.568974,34.482126,33.9588,33.4068,35.8747,36.8332,35.018375
26/01/2017 20:00,32.359850,34.117020,32.247231,32.549648,33.142986,33.014221,32.5864,31.9694,34.3195,34.9149,33.447550
26/01/2017 21:00,27.024776,30.924381,29.068047,29.203970,30.024324,29.805181,29.7851,28.7912,30.1627,30.5398,29.819700
26/01/2017 22:00,25.084933,27.033443,26.138130,25.693201,26.761005,26.406445,26.0611,24.7918,26.5028,26.8162,26.042975


In [80]:
PJM_DNN = PJM

In [81]:
PJM.drop(['DNN 2', 'DNN 3', 'DNN 4', 'DNN Ensemble', 'LEAR 84', 'LEAR 1092', 'LEAR 1456', 'LEAR Ensemble'], axis=1, inplace=True)
NP.drop(['DNN 2', 'DNN 3', 'DNN 4', 'DNN Ensemble', 'LEAR 84', 'LEAR 1092', 'LEAR 1456', 'LEAR Ensemble'], axis=1,  inplace=True)
FR.drop(['DNN 2', 'DNN 3', 'DNN 4', 'DNN Ensemble', 'LEAR 84', 'LEAR 1092', 'LEAR 1456', 'LEAR Ensemble'], axis=1,  inplace=True)

In [82]:
PJM

,Real price,DNN 1,LEAR 56
27/12/2016 00:00,19.113045,19.706123,20.1291
27/12/2016 01:00,18.042768,18.578808,19.5229
27/12/2016 02:00,17.587395,17.405716,19.0085
27/12/2016 03:00,17.399361,16.994461,18.9757
27/12/2016 04:00,17.804108,18.000519,19.3679
...,...,...,...
26/01/2017 19:00,32.532418,35.463852,33.9588
26/01/2017 20:00,32.359850,34.117020,32.5864
26/01/2017 21:00,27.024776,30.924381,29.7851
26/01/2017 22:00,25.084933,27.033443,26.0611


### PJM_LEAR_MAE_MAPE

In [83]:
MAE_PJM = pd.DataFrame(columns=['RMSE','MAE','sMAPE(%)'], index=PJM.index)
for date in PJM.index:
    MAE_PJM.loc[date]['RMSE'] = RMSE(pd.Series(PJM.loc[date]['LEAR 56']), pd.Series(PJM.loc[date]['Real price']))
    MAE_PJM.loc[date]['MAE'] = MAE(pd.Series(PJM.loc[date]['LEAR 56']), pd.Series(PJM.loc[date]['Real price']))
    MAE_PJM.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(PJM.loc[date]['LEAR 56']), pd.Series(PJM.loc[date]['Real price'])) * 100

In [84]:
MAE_PJM

,RMSE,MAE,sMAPE(%)
27/12/2016 00:00,1.016055,1.016055,5.178387
27/12/2016 01:00,1.480132,1.480132,7.880238
27/12/2016 02:00,1.421105,1.421105,7.766472
27/12/2016 03:00,1.576339,1.576339,8.667141
27/12/2016 04:00,1.563792,1.563792,8.413815
...,...,...,...
26/01/2017 19:00,1.426382,1.426382,4.290437
26/01/2017 20:00,0.22655,0.22655,0.697654
26/01/2017 21:00,2.760324,2.760324,9.717761
26/01/2017 22:00,0.976167,0.976167,3.817176


In [85]:
print(f"Overall RMSE : {RMSE(PJM['LEAR 56'], PJM['Real price'])}")

Overall RMSE : 3.057569933417542


In [86]:
print(f"Overall MAE : {MAE(PJM['LEAR 56'], PJM['Real price'])}")

Overall MAE : 2.2324875712365593


In [87]:
print(f"Overall sMAPE : {sMAPE(PJM['LEAR 56'], PJM['Real price']) * 100}")

Overall sMAPE : 7.597143505568613


### PJM_DNN_MAE_MAPE

In [88]:
MAE_PJM = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=PJM.index)
for date in PJM.index:
    MAE_PJM.loc[date]['RMSE'] = RMSE(pd.Series(PJM.loc[date]['DNN 1']), pd.Series(PJM.loc[date]['Real price']))
    MAE_PJM.loc[date]['MAE'] = MAE(pd.Series(PJM.loc[date]['DNN 1']), pd.Series(PJM.loc[date]['Real price']))
    MAE_PJM.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(PJM.loc[date]['DNN 1']), pd.Series(PJM.loc[date]['Real price'])) * 100

In [89]:
print(f"Overall MAE : {MAE(PJM['DNN 1'], PJM['Real price'])}")

Overall MAE : 2.1765672896236556


In [90]:
print(f"Overall sMAPE : {sMAPE(PJM['DNN 1'], PJM['Real price']) * 100}")

Overall sMAPE : 7.50924188506638


In [91]:
print(f"Overall RMSE : {RMSE(PJM['DNN 1'], PJM['Real price'])}")

Overall RMSE : 2.929825690654638


### PJM_NGBOOST_MAE_MAPE

In [92]:
NGB = pd.DataFrame(columns=['Predicted', 'Real Price'], index=pd.date_range(start=dt(2017, 12, 26, 0, 0, 0), end=dt(2018, 12, 24, 23, 0, 0), freq='H'))

In [93]:
temp = []
for _,row in enumerate(NGBoost.index):
    for value in NGBoost:
         temp.append(NGBoost.loc[row][value])

In [94]:
NGB['Predicted'] = temp

In [95]:
NGB.reset_index(inplace=True)
df.reset_index(inplace=True)
NGB['Real Price'] = df['Price']
NGB.set_index('index',inplace=True)
df.set_index('Date',inplace=True)

In [96]:
NGB = NGB[:30*24]

In [98]:
MAE_NGB = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=NGB.index)
for date in NGB.index:
    #MAE_NGB.loc[date]['RMSE'] = RMSE(pd.Series(NGB.loc[date]['Predicted']), pd.Series(PJM.loc[date]['Real price']))
    MAE_NGB.loc[date]['MAE'] = MAE(pd.Series(NGB.loc[date]['Predicted']), pd.Series(NGB.loc[date]['Real Price']))
    MAE_NGB.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(NGB.loc[date]['Predicted']), pd.Series(NGB.loc[date]['Real Price'])) * 100

In [20]:
#NGBOOST PJM MAE
print(f"Overall MAE : {MAE(NGB['Predicted'], NGB['Real Price'])}")

Overall MAE : 10.582150308006632


In [21]:
#NGBOOST PJM MAPE
print(f"Overall sMAPE : {sMAPE(NGB['Predicted'], NGB['Real Price']) * 100} %")

Overall sMAPE : 23.33853202166183 %


# NP LEAR, DNN, NGBOOST

In [99]:
NP

,Real price,DNN 1,LEAR 56
27/12/2016 00:00,24.08,24.530144,25.1290
27/12/2016 01:00,22.52,23.115845,24.2083
27/12/2016 02:00,20.13,22.563509,23.9289
27/12/2016 03:00,19.86,22.047380,23.4943
27/12/2016 04:00,20.09,23.055725,23.5104
...,...,...,...
26/01/2017 19:00,30.66,30.362480,30.3091
26/01/2017 20:00,29.85,29.554022,30.2290
26/01/2017 21:00,29.67,29.148659,29.6552
26/01/2017 22:00,29.05,28.654337,29.2686


### NP_LEAR_MAE_MAPE

In [100]:
 MAE_NP = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=NP.index)
for date in NP.index:
    MAE_PJM.loc[date]['RMSE'] = RMSE(pd.Series(NP.loc[date]['LEAR 56']), pd.Series(NP.loc[date]['Real price']))
    MAE_NP.loc[date]['MAE'] = MAE(pd.Series(NP.loc[date]['LEAR 56']), pd.Series(NP.loc[date]['Real price']))
    MAE_NP.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(NP.loc[date]['LEAR 56']), pd.Series(NP.loc[date]['Real price'])) * 100

In [101]:
print(f"Overall MAE : {MAE(NP['LEAR 56'], NP['Real price'])}")

Overall MAE : 1.7682907258064515


In [102]:
print(f"Overall sMAPE : {sMAPE(NP['LEAR 56'], NP['Real price']) * 100}")

Overall sMAPE : 5.3357347320699935


In [103]:
print(f"Overall RMSE : {RMSE(NP['LEAR 56'], NP['Real price'])}")

Overall RMSE : 3.184784680917026


### NP_DNN_MAE_MAPE

In [104]:
MAE_NP = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=NP.index)
for date in NP.index:
    MAE_NP.loc[date]['RMSE'] = RMSE(pd.Series(NP.loc[date]['DNN 1']), pd.Series(NP.loc[date]['Real price']))
    MAE_NP.loc[date]['MAE'] = MAE(pd.Series(NP.loc[date]['DNN 1']), pd.Series(NP.loc[date]['Real price']))
    MAE_NP.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(NP.loc[date]['DNN 1']), pd.Series(NP.loc[date]['Real price'])) * 100

In [105]:
print(f"Overall MAE : {MAE(NP['DNN 1'], NP['Real price'])}")

Overall MAE : 1.5872619102822583


In [106]:
print(f"Overall sMAPE : {sMAPE(NP['DNN 1'], NP['Real price']) * 100}")

Overall sMAPE : 4.714884901352434


In [107]:
print(f"Overall RMSE : {RMSE(NP['DNN 1'], NP['Real price'])}")

Overall RMSE : 2.903248470854296


### NP_NGBOOST_MAE_MAPE

In [108]:
NGBoost_NP = pd.read_csv('ngboost_NP.csv', index_col=[0])
NGBoost_NP_m = NGBoost_NP.drop(['h0_sd', 'h1_sd', 'h2_sd', 'h3_sd',
     'h4_sd', 'h5_sd', 'h6_sd', 'h7_sd',
       'h8_sd', 'h9_sd',  'h10_sd',  'h11_sd',
        'h12_sd',  'h13_sd',  'h14_sd', 
       'h15_sd',  'h16_sd',  'h17_sd',  'h18_sd',
        'h19_sd',  'h20_sd',  'h21_sd', 
       'h22_sd',  'h23_sd', 'RMSE'],axis=1)

In [109]:
df1 = pd.read_csv('df_testNP30.csv', index_col=[0])

In [110]:
df1

,Price,Exogenous 1,Exogenous 2
Date,,,
26/12/2017 00:00,25.82,41942,3817
26/12/2017 01:00,24.94,41236,3839
26/12/2017 02:00,23.81,41088,3796
26/12/2017 03:00,23.68,41089,3804
26/12/2017 04:00,24.29,41331,3829
...,...,...,...
25/01/2018 19:00,32.18,57273,1532
25/01/2018 20:00,31.49,55528,1500
25/01/2018 21:00,30.73,53916,1480


In [111]:
NGB_NP = pd.DataFrame(columns=['Predicted', 'Real Price'], index=pd.date_range(start=dt(2017, 12, 26, 0, 0, 0), end=dt(2018, 1, 25, 23, 0, 0), freq='H'))

In [112]:
NGB_NP

,Predicted,Real Price
2017-12-26 00:00:00,NaN,NaN
2017-12-26 01:00:00,NaN,NaN
2017-12-26 02:00:00,NaN,NaN
2017-12-26 03:00:00,NaN,NaN
2017-12-26 04:00:00,NaN,NaN
...,...,...
2018-01-25 19:00:00,NaN,NaN
2018-01-25 20:00:00,NaN,NaN
2018-01-25 21:00:00,NaN,NaN
2018-01-25 22:00:00,NaN,NaN


In [113]:
temp1 = []
for _,row in enumerate(NGBoost_NP_m.index):
    for value in NGBoost_NP_m:
         temp1.append(NGBoost_NP_m.loc[row][value])

In [114]:
temp1

[23.3096005,
 23.37788854,
 23.37788854,
 23.37788854,
 23.37788854,
 23.61272994,
 24.19558279,
 24.59846164,
 25.17615885,
 25.29089009,
 25.33970108,
 25.28894334,
 25.28894334,
 25.28894334,
 25.28894334,
 25.84064117,
 25.97547832,
 25.97311673,
 25.84064117,
 25.53141067,
 25.25526963,
 25.13279607,
 24.56989233,
 24.56255672,
 24.12914853,
 24.54241854,
 26.34956771,
 27.30961637,
 27.64820247,
 28.24267722,
 29.18921027,
 30.67432156,
 31.30762269,
 31.30762269,
 31.52160835,
 31.54929989,
 31.4982423,
 31.47125416,
 31.59806086,
 32.54148872,
 33.25341913,
 33.28040727,
 33.42359747,
 32.68081562,
 32.76383801,
 32.06855064,
 31.70829604,
 31.37052362,
 30.16143934,
 30.36758571,
 30.37627394,
 30.41069009,
 30.40200186,
 30.45166677,
 31.64345132,
 33.14186172,
 33.28095011,
 33.35899584,
 33.73967862,
 33.5865511,
 33.19041674,
 33.19041674,
 33.68892088,
 34.04301913,
 34.84450591,
 34.13552788,
 32.54148872,
 31.54929989,
 31.13509735,
 30.43980998,
 29.82214758,
 29.01086

In [115]:
NGB_NP['Predicted'] = temp1

ValueError: Length of values (912) does not match length of index (744)

In [147]:
NGB_NP.reset_index(inplace=True)
df1.reset_index(inplace=True)
NGB_NP['Real Price'] = df1['Price']
NGB_NP.set_index('index',inplace=True)
df1.set_index('Date',inplace=True)

In [148]:
MAE_NGB_NP = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=NGB_NP.index)
for date in NGB_NP.index:
    MAE_NGB_NP.loc[date]['MAE'] = MAE(pd.Series(NGB_NP.loc[date]['Predicted']), pd.Series(NGB_NP.loc[date]['Real Price']))
    MAE_NGB_NP.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(NGB_NP.loc[date]['Predicted']), pd.Series(NGB_NP.loc[date]['Real Price'])) * 100

In [149]:
#NGBOOST PJM MAE
print(f"Overall MAE : {MAE(NGB_NP['Predicted'], NGB_NP['Real Price'])}")

Overall MAE : nan


In [150]:
#NGBOOST PJM MAPE
print(f"Overall sMAPE : {sMAPE(NGB_NP['Predicted'], NGB_NP['Real Price']) * 100} %")

Overall sMAPE : nan %


# FR LEAR, DNN, NGBOOST


In [119]:
FR

,Real price,DNN 1,LEAR 56
04/01/2015 00:00,36.26,37.908104,38.9034
04/01/2015 01:00,32.28,34.774220,33.6675
04/01/2015 02:00,27.63,30.165806,29.7656
04/01/2015 03:00,23.94,24.531939,25.5819
04/01/2015 04:00,15.69,22.675518,22.3733
...,...,...,...
03/02/2015 19:00,68.71,75.608505,61.6084
03/02/2015 20:00,61.75,62.047821,55.5862
03/02/2015 21:00,53.17,51.751099,49.9943
03/02/2015 22:00,51.80,52.766350,50.5059


### FR_LEAR_MAE_MAPE

In [118]:
MAE_FR = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=FR.index)
for date in FR.index:
    MAE_FR.loc[date]['RMSE'] = RMSE(pd.Series(FR.loc[date]['LEAR 56']), pd.Series(NP.loc[date]['Real price']))
    MAE_FR.loc[date]['MAE'] = MAE(pd.Series(FR.loc[date]['LEAR 56']), pd.Series(FR.loc[date]['Real price']))
    MAE_FR.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(FR.loc[date]['LEAR 56']), pd.Series(FR.loc[date]['Real price'])) * 100

KeyError: '04/01/2015 00:00'

In [120]:
MAE_FR

,MAE,sMAPE(%)
04/01/2015 00:00,NaN,NaN
04/01/2015 01:00,NaN,NaN
04/01/2015 02:00,NaN,NaN
04/01/2015 03:00,NaN,NaN
04/01/2015 04:00,NaN,NaN
...,...,...
03/02/2015 19:00,NaN,NaN
03/02/2015 20:00,NaN,NaN
03/02/2015 21:00,NaN,NaN
03/02/2015 22:00,NaN,NaN


In [37]:
print(f"Overall MAE : {MAE(FR['LEAR 56'], FR['Real price'])}")

Overall MAE : 3.6409138440860214


In [38]:
print(f"Overall sMAPE : {sMAPE(FR['LEAR 56'], FR['Real price']) * 100} %")

Overall sMAPE : 10.136066936173954 %


In [ ]:
print(f"Overall RMSE : {RMSE(FR['LEAR 56'], FR['Real price'])}")

### FR_DNN_MAE_MAPE

In [73]:
MAE_FR = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=FR.index)
for date in FR.index:
    MAE_FR.loc[date]['RMSE'] = RMSE(pd.Series(FR.loc[date]['DNN 1']), pd.Series(NP.loc[date]['Real price']))
    MAE_FR.loc[date]['MAE'] = MAE(pd.Series(FR.loc[date]['DNN 1']), pd.Series(FR.loc[date]['Real price']))
    MAE_FR.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(FR.loc[date]['DNN 1']), pd.Series(FR.loc[date]['Real price'])) * 100

KeyError: '04/01/2015 00:00'

In [40]:
print(f"Overall MAE : {MAE(FR['DNN 1'], FR['Real price'])}")

Overall MAE : 3.492987625467742


In [41]:
print(f"Overall sMAPE : {sMAPE(FR['DNN 1'], FR['Real price']) * 100} %")

Overall sMAPE : 9.197918358127502 %


### FR_NGBOOST_MAE_MAPE

In [52]:
MM  = pd.read_csv('FR_DATA.csv', index_col=[0])

In [53]:
MM

,Predicted,Real Price
Date,,
01/12/2016 00:00,44.463552,72.00
01/12/2016 01:00,39.963504,65.80
01/12/2016 02:00,38.171424,59.99
01/12/2016 03:00,34.852367,50.69
01/12/2016 04:00,32.619534,52.58
...,...,...
31/12/2016 19:00,50.009299,69.72
31/12/2016 20:00,48.912822,57.10
31/12/2016 21:00,44.061156,54.49


In [55]:
MAE_NGB_FR = pd.DataFrame(columns=['MAE','sMAPE(%)'], index=MM.index)
for date in MM.index:
    MM.loc[date]['MAE'] = MAE(pd.Series(MM.loc[date]['Predicted']), pd.Series(MM.loc[date]['Real Price']))
    MM.loc[date]['sMAPE(%)'] = sMAPE(pd.Series(MM.loc[date]['Predicted']), pd.Series(MM.loc[date]['Real Price'])) * 100

In [57]:
#NGBOOST FR MAE
print(f"Overall MAE : {MAE(MM['Predicted'], MM['Real Price'])}")

Overall MAE : 14.890192014395161


In [58]:
#NGBOOST FR MAPE
print(f"Overall sMAPE : {sMAPE(MM['Predicted'], MM['Real Price']) * 100} %")

Overall sMAPE : 28.618586036865622 %
